## Import Required Libraries

In [53]:
import pandas as pd
import numpy as np
from numpy.random import normal
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

## Import data and inspect

In [54]:
df = pd.read_csv('test.csv')

In [55]:
df['Embarked'].value_counts().keys()[0]

'S'

In [56]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [6]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


## Processing Titles

In [57]:
df['Titles'] = df['Name'].str.extract('(\w*\.)')

In [58]:
df['Titles'].value_counts()

Mr.        240
Miss.       78
Mrs.        72
Master.     21
Rev.         2
Col.         2
Ms.          1
Dr.          1
Dona.        1
Name: Titles, dtype: int64

In [59]:
df.loc[~df['Titles'].isin(['Mr.', 'Miss.', 'Mrs.', 'Master.']), 'Titles'] = 'Other'

In [60]:
df['Titles'].value_counts()

Mr.        240
Miss.       78
Mrs.        72
Master.     21
Other        7
Name: Titles, dtype: int64

In [11]:
df = pd.concat([df, pd.get_dummies(df['Titles'])], axis=1)

## Processing Age Related Data

In [12]:
df['Age'].isnull().sum()

177

There are 177 null values - we have two choices here: impute the data using the mean/median OR randomly assign a value. To randomly assign a value we will use the Numpy Random function passing in the index/passenger ID as the seed for reproducibility.

In [100]:
df['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

#### Impute Age Using Randomly Assigned Value from Mean/Std (Option 1)

In [101]:
seed = np.random.RandomState(15144)
age_estimates = seed.normal(df['Age'].mean(), scale=df['Age'].std(), size=df['Age'].isnull().sum())

In [102]:
age_estimates = [x if x > 0 else df['Age'].mean() for x in age_estimates]
df.loc[df['Age'].isnull(), 'Age'] = age_estimates

In [103]:
df['Age'].describe()

count    891.000000
mean      29.549299
std       14.262134
min        0.420000
25%       20.000000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

#### Imputing Age Using Median/Mean (Option 2)

In [10]:
mean_age_by_gender = df.groupby('Sex').mean()['Age'].to_dict()
null_age = df.loc[df['Age'].isna(), :].Sex.map(mean_age_by_gender)
df.loc[null_age.index, 'Age'] = null_age

## Processing Ticket Related Data

In [61]:
df['Ticket'] = df['Ticket'].str.replace('\.|/', '')

In [62]:
temp_ticket = df['Ticket'].str.extract('(\w*)?\s?(.*)')

In [63]:
for index, row in temp_ticket.iterrows():
    
    if not row[1]:
        
        temp_ticket.iloc[index][1] = row[0]
        temp_ticket.iloc[index][0] = ''

temp_ticket = pd.concat([temp_ticket[1], 
                         pd.get_dummies(temp_ticket[0])[temp_ticket[0].value_counts().keys()[1:10].tolist()]], 
                        axis=1)

In [64]:
temp_ticket[1] = temp_ticket[1].str.extract('(\d{2,})')
temp_ticket[1] = temp_ticket[1].fillna(0).astype(int)
temp_ticket.rename(columns={1:"TicketID"}, inplace=True)

## Mining Cabin Related Data

#### Extracting Cabin Info into Numerical Values

In [65]:
cabin_extract = df['Cabin'].str.extract('(\w)\d*$')
cabin_extract = pd.get_dummies(cabin_extract[0])
z = dict(zip(cabin_extract.columns.tolist(), [x + '_CABIN' for x in cabin_extract.columns.tolist()]))
cabin_extract.rename(columns=z, inplace=True)
df = pd.concat([df, cabin_extract], axis=1)


#cabin_extract = pd.concat([cabin_extract[1], pd.get_dummies(cabin_extract[0])], axis=1).fillna(0)

In [66]:
cabin_extract

,A_CABIN,B_CABIN,C_CABIN,D_CABIN,E_CABIN,F_CABIN,G_CABIN
0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0


In [19]:
cabin_extract.rename(columns={1:'CabinNo'}, inplace=True)
z = dict(zip(cabin_extract.columns[1:].tolist(), [x + '_CABIN' for x in cabin_extract.columns[1:].tolist()]))
cabin_extract.rename(columns=z, inplace=True)
cabin_extract.loc[cabin_extract['CabinNo'] == '', 'CabinNo'] = 0
cabin_extract.loc[:, 'CabinNo'] = cabin_extract['CabinNo'].astype(int, errors='ignore')
cabin_extract.head()

KeyError: 'CabinNo'

In [67]:
df = pd.concat([df, cabin_extract], axis=1)

#### Creating Boolean Cabin Value

In [68]:
df.loc[:, 'InCabin'] = ~df['Cabin'].isna()

## Passenger Class Encoded

In [69]:
pclass = pd.get_dummies(df['Pclass'])

In [70]:
map = dict(zip(pclass.columns.tolist(), ['class_' + str(x) for x in pclass.columns.tolist()]))

In [71]:
pclass.rename(columns=map, inplace=True)

In [72]:
class_dummies = pd.get_dummies(df['Pclass'])
class_dummies_names = ['class_' + str(x) for x in class_dummies.columns.to_list()]
df[class_dummies_names] = class_dummies

## Embarked Encoded

In [73]:
df = pd.concat([df, pd.get_dummies(df['Embarked'])], axis=1)

## Gender Encoded

In [74]:
df[['female', 'male']] = pd.get_dummies(df['Sex'])

## Feature Engineering

In [116]:
# TO DO -- 

In [117]:
#df.loc[:, 'Child'] = df['Age'] <= 12

#Life expectancy in 1912 was ~53, well say someone is elderly if they are older than 60
#df.loc[:, 'Elderly'] = df['Age'] >= 60

In [75]:
df['FamilyAboard'] = df['Parch'] + df['SibSp']

In [76]:
df['IsAlone'] = df['FamilyAboard'] == 0

# -Age Imputing-

In [77]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,class_1,class_2,class_3,C,Q,S,female,male,FamilyAboard,IsAlone
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,...,0,0,1,0,1,0,0,1,0,True
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,...,0,0,1,0,0,1,1,0,1,False
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,...,0,1,0,0,1,0,0,1,0,True
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,...,0,0,1,0,0,1,0,1,0,True
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,...,0,0,1,0,0,1,1,0,2,False


In [31]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Titles', 'Master.',
       'Miss.', 'Mr.', 'Mrs.', 'Other', 'A_CABIN', 'B_CABIN', 'C_CABIN',
       'D_CABIN', 'E_CABIN', 'F_CABIN', 'G_CABIN', 'T_CABIN', 'A_CABIN',
       'B_CABIN_CABIN', 'C_CABIN_CABIN', 'D_CABIN_CABIN', 'E_CABIN_CABIN',
       'F_CABIN_CABIN', 'G_CABIN_CABIN', 'T_CABIN_CABIN', 'InCabin', 'class_1',
       'class_2', 'class_3', 'C', 'Q', 'S', 'female', 'male', 'FamilyAboard',
       'IsAlone'],
      dtype='object')

In [78]:
df.groupby(['Sex', 'Pclass','Titles']).count()

PassengerId  Name  Age  SibSp  Parch  Ticket  Fare  \
Sex    Pclass Titles                                                        
female 1      Miss.             14    14   14     14     14      14    14   
              Mrs.              35    35   33     35     35      35    35   
              Other              1     1    1      1      1       1     1   
       2      Miss.             16    16   16     16     16      16    16   
              Mrs.              14    14   13     14     14      14    14   
       3      Miss.             48    48   34     48     48      48    48   
              Mrs.              23    23   16     23     23      23    23   
              Other              1     1    0      1      1       1     1   
male   1      Master.            2     2    2      2      2       2     2   
              Mr.               52    52   45     52     52      52    52   
              Other              3     3    3      3      3       3     3   
       2      Master.            2     2    2      2      2       2     2   
              Mr.               59    59   55     59     59      59    59   
              Other              2     2    2      2      2       2     2   
       3      Master.           17    17   13     17     17      17    17   
              Mr.              129   129   83    129    129     129   128   

                       Cabin  Embarked  A_CABIN  ...  class_1  class_2  \
Sex    Pclass Titles                             ...                     
female 1      Miss.        9        14       14  ...       14       14   
              Mrs.        30        35       35  ...       35       35   
              Other        1         1        1  ...        1        1   
       2      Miss.        1        16       16  ...       16       16   
              Mrs.         2        14       14  ...       14       14   
       3      Miss.        1        48       48  ...       48       48   
              Mrs.         0        23       23  ...       23       23   
              Other        0         1        1  ...        1        1   
male   1      Master.      2         2        2  ...        2        2   
              Mr.         35        52       52  ...       52       52   
              Other        3         3        3  ...        3        3   
       2      Master.      0         2        2  ...        2        2   
              Mr.          4        59       59  ...       59       59   
              Other        0         2        2  ...        2        2   
       3      Master.      0        17       17  ...       17       17   
              Mr.          3       129      129  ...      129      129   

                       class_3    C    Q    S  female  male  FamilyAboard  \
Sex    Pclass Titles                                                        
female 1      Miss.         14   14   14   14      14    14            14   
              Mrs.          35   35   35   35      35    35            35   
              Other          1    1    1    1       1     1             1   
       2      Miss.         16   16   16   16      16    16            16   
              Mrs.          14   14   14   14      14    14            14   
       3      Miss.         48   48   48   48      48    48            48   
              Mrs.          23   23   23   23      23    23            23   
              Other          1    1    1    1       1     1             1   
male   1      Master.        2    2    2    2       2     2             2   
              Mr.           52   52   52   52      52    52            52   
              Other          3    3    3    3       3     3             3   
       2      Master.        2    2    2    2       2     2             2   
              Mr.           59   59   59   59      59    59            59   
              Other          2    2    2    2       2     2             2   
       3      Master.       17   17   17   17      17    17            

In [79]:
t = df.groupby(['Sex', 'Pclass','Titles']).mean()['Age'].reset_index().set_index(['Sex', 'Pclass', 'Titles'])

In [52]:
t.to_csv('ages.csv', index=True)

In [89]:
t.loc['female']['Age'].mean()

30.878897013350798

In [86]:
df.loc[df['Age'].isna(), ['Sex', 'Pclass', 'Titles']].drop_duplicates()

,Sex,Pclass,Titles
10,male,3,Mr.
22,female,1,Mrs.
33,female,3,Mrs.
36,female,3,Miss.
41,male,1,Mr.
54,male,2,Mr.
65,female,2,Mrs.
88,female,3,Other
244,male,3,Master.


In [50]:
for index, row in df.loc[df['Age'].isna(), :].iterrows():
    
    print(index)
    #df.loc[index, 'Age'] = t.loc[(row['Sex'], row['Pclass'], row['Titles']), 'Age']

## Dropping unecessary data

In [46]:
df = df.drop(df[df['Embarked'].isna()].index)
drop_columns = ['PassengerId', 'Sex', 'Ticket', 'Cabin', 'Name', 'Pclass', 'Embarked', 'Titles']
df_dropped = df.drop(drop_columns, axis=1)

## Export Processed Data to Cleaned CSV

In [47]:
df_dropped.to_csv('train_cleaned.csv', index=False)